In [3]:
import pandas as pd
import requests
import urllib.parse
import geopy.distance
import numpy as np

In [4]:

df1 = pd.read_csv('stations.csv')
# df1.head(3)

df1=df1[['#station']]


In [5]:
finalDic={}

for index, row in df1.iterrows():
   currStation=row['#station']
   if currStation not in finalDic.keys():
       finalDic[currStation]=1
   else:
        finalDic[currStation]+=1

In [6]:
df2 = pd.read_csv('stations2.csv')
df2.head(3)

,#line,#station_B,accSum
0,Main Line,Fort,30
1,Main Line,Maradana,23
2,Main Line,Kelaniya,4


In [7]:
for index, row in df2.iterrows():
   currStation=row['#station_B']
   if currStation not in finalDic.keys():
       finalDic[currStation]=1
   else:
        finalDic[currStation]+=row['accSum']

In [8]:
newDic={'station':[],'countAccidents':[]}

for key,value in finalDic.items():
       newDic['station'].append(key)
       newDic['countAccidents'].append(value)
df3=pd.DataFrame.from_dict(newDic)
df3.head(3)

,station,countAccidents
0,Fort,32
1,Maradana,24
2,Dematagoda,1


In [9]:
df3['lat']=None
df3['lon']=None

for i in range(len(df3)):
 
  address = df3.iat[i,0]+' railway station, sri lanka'
  
  try:
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    cod1=response[0]["lat"]
    cod2=response[0]["lon"]

    # result=gmaps_key.geocode(address)
    # cod1=result[0]['geometry']['location']['lat']
    # cod2=result[0]['geometry']['location']['lat']
    # print(cod1,cod2)
    df3.iat[i,df3.columns.get_loc("lat")]=cod1
    df3.iat[i,df3.columns.get_loc("lon")]=cod2
  except:
    pass
 
  if i%15==0:
    
     print(df3.loc[[i]])
      
df3.info()
df3.to_csv('allTrainStationLocationsUnique.csv', index=False)
   

  station  countAccidents        lat        lon
0    Fort              32  6.9340391  79.850372
     station  countAccidents        lat         lon
15  Daraluwa              17  7.1070543  80.0104022
       station  countAccidents   lat   lon
30  Buthgamuwa               1  None  None
     station  countAccidents        lat         lon
45  Koshinna              16  7.2335885  80.5950939
         station  countAccidents   lat   lon
60  Greatwestern               1  None  None
     station  countAccidents        lat         lon
75  Demodara               1  6.9028736  81.0629186
    station  countAccidents      lat         lon
90  Kandana              17  7.04929  79.9015367
      station  countAccidents   lat   lon
105  Waikkala               1  None  None
             station  countAccidents   lat   lon
120  Arachchikattuwa              21  None  None
             station  countAccidents   lat   lon
135  Talawattegedara               1  None  None
           station  countAccidents    

In [10]:
df3.head(3)

,station,countAccidents,lat,lon
0,Fort,32,6.9340391,79.850372
1,Maradana,24,6.9292373,79.865796
2,Dematagoda,1,6.9374165,79.8790233


In [11]:
dfTrain = pd.read_csv('trainPaths.csv')
dfTrain.dtypes

lat    float64
lon    float64
dtype: object

In [12]:
df4=df3.copy()
df4=df4.dropna()
df4.reset_index(drop=True, inplace=True)
df4.head()
df4['lat'] = df4['lat'].astype(float)
df4['lon'] = df4['lon'].astype(float)

df4.dtypes

df4['crackCount']=[0]*len(df4)
df4.head()

,station,countAccidents,lat,lon,crackCount
0,Fort,32,6.934039,79.850372,0
1,Maradana,24,6.929237,79.865796,0
2,Dematagoda,1,6.937417,79.879023,0
3,Kelaniya,5,6.961059,79.894473,0
4,Wanawasala,1,6.975569,79.899258,0


In [13]:

def getCrackcount(val):
    if val>0:
       return val*2
    return 1

rowCount=0
for i in range(len(df4)):
  count=0
  val = df4.iat[i,1]
  latDis=df4.iat[i,2]
  lonDis=df4.iat[i,3]

  crackVal=getCrackcount(val)
  df4.iat[i,4]=crackVal
  # allList=[]
  # realCracks=0
  for j in range(len(dfTrain)):  
   

    lat,lon = dfTrain.iat[j,0],dfTrain.iat[j,1]
    dist=geopy.distance.geodesic((lat,lon), (latDis,lonDis)).m
    if dist<1300 and (lat not in df4['lat'].values or lat not in df4['lon'].values ):
              # if dist<1300:
              #    realCracks+=1
              newRow={'station':[df4.iat[i,0]] , 'countAccidents': [val], 'lat': [lat],'lon':[lon],'crackCount': [crackVal]}
              # allList.append([dist,newRow])
              # df4 = df4.append(newRow, ignore_index=True)
              df4 = pd.concat([df4,pd.DataFrame(newRow)], ignore_index = True)
              if rowCount%20==0:
                print("row Added",rowCount)
              rowCount+=1
              count+=1
    if count==25:
        print("breaked")
        break
  
  # allList=sorted(allList, key=lambda item:item[0])
  
  # realCracks= realCracks if realCracks<=15 else 15
  # for ind,value in enumerate(allList):
  #       crackCount=0
  #       if realCracks>0:
  #            crackCount=crackVal if ind<realCracks else 0
        
  #       newRow=value[1]
  #       newRow['crackCount']=[int(crackCount)]
  #       df4 = pd.concat([df4,pd.DataFrame(newRow)], ignore_index = True)

        # df4 = df4.append(newRow, ignore_index=True)



df4.reset_index()           
     
      

row Added 0
row Added 20
breaked
row Added 40
breaked
row Added 60
breaked
row Added 80
breaked
row Added 100
row Added 120
breaked
row Added 140
breaked
row Added 160
breaked
row Added 180
breaked
row Added 200
row Added 220
breaked
row Added 240
breaked
row Added 260
breaked
row Added 280
breaked
row Added 300
row Added 320
breaked
row Added 340
breaked
row Added 360
breaked
row Added 380
breaked
row Added 400
row Added 420
breaked
row Added 440
breaked
row Added 460
breaked
row Added 480
breaked
row Added 500
row Added 520
breaked
row Added 540
breaked
row Added 560
row Added 580
breaked
row Added 600
breaked
row Added 620
breaked
row Added 640
breaked
row Added 660
row Added 680
breaked
row Added 700
breaked
row Added 720
breaked
row Added 740
breaked
row Added 760
row Added 780
breaked
row Added 800
breaked
row Added 820
breaked
row Added 840
breaked
row Added 860
row Added 880
breaked
row Added 900
breaked
row Added 920
breaked
row Added 940
breaked
row Added 960
row Added 980
br

,index,station,countAccidents,lat,lon,crackCount
0,0,Fort,32,6.934039,79.850372,64
1,1,Maradana,24,6.929237,79.865796,48
2,2,Dematagoda,1,6.937417,79.879023,2
3,3,Kelaniya,5,6.961059,79.894473,10
4,4,Wanawasala,1,6.975569,79.899258,2
...,...,...,...,...,...,...
3628,3628,Walgama,1,5.945658,80.510257,2
3629,3629,Walgama,1,5.945487,80.509554,2
3630,3630,Walgama,1,5.945124,80.508294,2
3631,3631,Walgama,1,5.944787,80.507167,2


In [14]:
df4.info

<bound method DataFrame.info of          station  countAccidents       lat        lon  crackCount
0           Fort              32  6.934039  79.850372          64
1       Maradana              24  6.929237  79.865796          48
2     Dematagoda               1  6.937417  79.879023           2
3       Kelaniya               5  6.961059  79.894473          10
4     Wanawasala               1  6.975569  79.899258           2
...          ...             ...       ...        ...         ...
3628     Walgama               1  5.945658  80.510257           2
3629     Walgama               1  5.945487  80.509554           2
3630     Walgama               1  5.945124  80.508294           2
3631     Walgama               1  5.944787  80.507167           2
3632     Walgama               1  5.944712  80.506635           2

[3633 rows x 5 columns]>

In [15]:
df5=df4.drop(['station','countAccidents'], axis=1)
df5.to_csv('trainCracksFinalData.csv', index=False)